In [ ]:
!pip install -q google-genai

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set


# Exercise 6: The Agentic Loop

In this exercise, we put it all together into an **agent loop** — the core pattern behind AI agents.

By the end, you'll understand:
- How to build a loop that keeps running until the model stops requesting tools
- How to dispatch tool calls to actual functions
- How the model can chain **multiple** tool calls to answer a single question

## Step 1: Setup

In [ ]:
from google import genai
from google.genai import types
from datetime import datetime

client = genai.Client()

## Step 2: Define multiple tools and a dispatcher

This time we have **two** tools. The `call_tool` function maps tool names to their implementations — this is how real agents work.

In [ ]:
def get_weather(location: str) -> str:
    """Get the current weather in a given location."""
    pass


def get_time() -> str:
    """Get the current date and time."""
    pass


def call_tool(tool_name, tool_input):
    if tool_name == "get_weather":
        return f"72°F and sunny in {tool_input['location']}"
    elif tool_name == "get_time":
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    else:
        return "Tool not found."


config = types.GenerateContentConfig(
    tools=[get_weather, get_time],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True),
)

## Step 3: The agent loop

This is the core pattern. The loop:
1. Sends contents to the model
2. Checks if the response contains any `function_call` parts
3. If yes — executes each tool, appends results, and loops back to step 1
4. If no — the model is done, we print the final text and break

Because we ask about **both** weather and time, the model needs to call two tools — watch the loop iterate!

In [ ]:
contents = [types.Content(role="user", parts=[types.Part.from_text(text="What's the weather and current time in Tokyo?")])]

loop_count = 0

while True:
    loop_count += 1
    print(f"--- Loop iteration {loop_count} ---")

    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=contents,
        config=config,
    )

    model_content = response.candidates[0].content
    contents.append(model_content)

    function_calls = [part.function_call for part in model_content.parts if part.function_call]

    if function_calls:
        tool_response_parts = []
        for fc in function_calls:
            tool_result = call_tool(fc.name, dict(fc.args))
            print(f"  Tool: {fc.name}({dict(fc.args)}) → {tool_result}")
            tool_response_parts.append(
                types.Part.from_function_response(name=fc.name, response={"output": tool_result})
            )
        contents.append(types.Content(role="user", parts=tool_response_parts))
    else:
        print(f"\n{response.text}")
        print("\nDone! No more tool calls.")
        break

## Step 4: Review the full message history

In [ ]:
for msg in contents:
    print(f"--- {msg.role} ---")
    for part in msg.parts:
        if part.text:
            print(part.text)
        elif part.function_call:
            print(f"[function_call: {part.function_call.name}({part.function_call.args})]")
        elif part.function_response:
            print(f"[function_response: {part.function_response.name} → {part.function_response.response}]")
    print()

## Key takeaway

This is the **agentic loop** — the foundation of every AI agent:

```
while True:
    response = call_model(contents)
    if response has function calls:
        execute tools, append results
    else:
        break  # model is done
```

Everything else — multiple tools, error handling, human-in-the-loop, memory — is built on top of this pattern.

Now let's build some real agents!